![](https://images.aicrowd.com/raw_images/challenges/banner_file/1023/a8ebe297c369ea4a7697.png)

<h2><center>Starter Code for Sentiment Classification</center></h2>


In this baseline we will be training an sklearn model to do a multi-class classificattion of sentiment from face embeddings. 

## Downloading Dataset

Installing puzzle datasets via `aicrowd-cli`

In [1]:
# !pip install aicrowd-cli

# Make sure to re-run below code whenever you restart colab notebook
# %load_ext aicrowd.magic

In [2]:
# # Logging in from our AIcrowd account. Make sure you have accepted the puzzle rules before logging in!  

# %aicrowd login

In [3]:
# # Creating a new data directory and downloading the dataset 

# !rm -rf data
# !mkdir data
# %aicrowd ds dl -c sentiment-classification -o data

## Importing Libraries

In this baseline, we will be sing sklearn [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify the sentiment of face embeddings. 

In [4]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

random.seed(42)

## Reading Dataset

As mented in the challenge readme, we have three different sets provided - train, validation and test respectively.  

In [5]:
# Readging the csv 

train = pd.read_csv("data/train.csv")
val = pd.read_csv("data/val.csv")
submission = pd.read_csv("data/sample_submission.csv")

train

,embeddings,label
0,"[0.3206779360771179, 0.988215982913971, 1.0441...",positive
1,"[0.05074610561132431, 1.0742985010147095, 0.60...",negative
2,"[0.41962647438049316, 0.4505457878112793, 1.39...",negative
3,"[0.4361684024333954, 0.19191382825374603, 0.83...",positive
4,"[0.6382085084915161, 0.8352395296096802, 0.393...",neutral
...,...,...
4995,"[2.2057647705078125, 1.1072001457214355, 0.435...",neutral
4996,"[0.6344252228736877, 1.164398193359375, 0.7155...",negative
4997,"[0.9160683155059814, 0.39996421337127686, 0.82...",negative
4998,"[0.006456990726292133, 0.18667978048324585, 0....",positive


In [6]:
# Getting the feature and labels from each set. 


X = np.array([literal_eval(embedding)  for embedding in train['embeddings'].values])
y = np.array(train['label'].values)

X_val = np.array([literal_eval(embedding)  for embedding in val['embeddings'].values])
y_val = np.array(val['label'].values)

In [7]:
X.shape, y.shape, X_val.shape, y.shape

((5000, 512), (5000,), (2000, 512), (5000,))

## Training the model

Here, we will be training our model using the training set. 

In [8]:
model = RandomForestClassifier()
model

RandomForestClassifier()

In [9]:
model.fit(X, y)

RandomForestClassifier()

### Testing the Model

Here, we will be evaluator our model using validation set 

In [10]:
y_pred = model.predict(X_val)

print(f"F1 Score : {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy Score : {accuracy_score(y_val, y_pred)}")

F1 Score : 0.6729124468148805
Accuracy Score : 0.6765


# TF Keras

In [11]:
import tensorflow as tf

In [12]:
from datetime import datetime

In [13]:
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback, TensorBoardCallback


In [14]:
y[y=='positive']=2
y[y=='negative']=0
y[y=='neutral']=1
y = tf.one_hot(y,3)
y[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

In [15]:
np.unique(y,return_counts=True)

(array([0., 1.], dtype=float32), array([10000,  5000]))

In [16]:
y_val[y_val=='positive']=2
y_val[y_val=='negative']=0
y_val[y_val=='neutral']=1
y_val = tf.one_hot(y_val,3)
y_val[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)>

In [29]:
def my_f1(y_val, y_pred, sample_weight=None):
    return f1_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1),average='weighted')

def my_acc(y_val, y_pred, sample_weight=None):
    return accuracy_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1))

def getModel(neurons0=1024,neurons1=1024,neurons2=1024, layers1=3, layers2=3, dropout=0.5, lr=3e-3, ln=0, noise=1):
    ls=[tf.keras.layers.Dense(neurons0, activation='relu', input_shape=(512,))]
    
    if noise>0:
        ls.append(tf.keras.layers.GaussianNoise(0.1))
    
    if ln>0:
        ls.append(tf.keras.layers.LayerNormalization())
        
    for _ in range(layers1):
        ls.append(tf.keras.layers.Dense(neurons1, activation='relu'),)
    for _ in range(layers2):
        ls.append(tf.keras.layers.Dense(neurons2, activation='relu'),)
    if ln>1:
        ls.append(tf.keras.layers.LayerNormalization())
    ls.append(tf.keras.layers.Dropout(dropout))
    ls.append(tf.keras.layers.Dense(3, activation='softmax'))
    model = tf.keras.models.Sequential(ls)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['mae','mse','accuracy',my_f1,my_acc],
                  run_eagerly=True)
    return model

# def getModelCls():
#     model = tf.keras.models.Sequential([tf.keras.layers.Dense(1024, activation='relu', input_shape=(512,)),
# #                                         tf.keras.layers.Dropout(0.25),
#                                         tf.keras.layers.LayerNormalization(),
#                                         tf.keras.layers.Dense(1024, activation='relu'),
#                                         tf.keras.layers.Dense(1024, activation='relu'),
#                                         tf.keras.layers.Dropout(0.5),
#                                         tf.keras.layers.Dense(1024, activation='relu'),
#                                         tf.keras.layers.LayerNormalization(),
#                                         tf.keras.layers.Dropout(0.5),
#                                         tf.keras.layers.Dense(3, activation='softmax')])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#                   loss='categorical_crossentropy',
#                   metrics=['mae','mse','accuracy',my_f1,my_acc],
#                   run_eagerly=True)
#     return model
# model = getModelCls()
# model.summary()

# model.fit(X/10.0,y,
#           validation_data=(X_val/10.0,y_val),
#           batch_size=1024,
#           epochs=500,
#           verbose=0,
#          callbacks=[tf.keras.callbacks.TensorBoard(
#     log_dir="logs/0/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
#     histogram_freq=1,
#     write_graph=True,
#     update_freq="epoch"),#PlotLossesKerasTF(),
# #                    tf.keras.callbacks.ModelCheckpoint(
# #     'checkpoint',
# #     monitor="val_my_f1",
# #     verbose=1,
# #     save_best_only=True,
# #     save_weights_only=False,
# #     mode="max",
# #     save_freq="epoch",),
#                     tf.keras.callbacks.EarlyStopping(
#                                     monitor="val_my_f1",
#                                     patience=100,
#                                     verbose=1,
#                                     mode="max",
#                                     restore_best_weights=True,
#                                 ),
#                    tf.keras.callbacks.ReduceLROnPlateau(
#     monitor="val_my_f1",
#     factor=0.3,
#     patience=10,
#     verbose=1,
#     mode="max",
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=1e-10,
# )])

In [30]:
max_epochs=100
def objective(trial):
    neurons0 = trial.suggest_int("neurons0", 16, 2048, log=True)
    neurons1 = trial.suggest_int("neurons1", 16, 2048, log=True)
    neurons2 = trial.suggest_int("neurons2", 16, 2048, log=True)
    lr = trial.suggest_float("lr", 1e-7, 1e-1, log=True)
    dropout = trial.suggest_float("dropout", 0, 0.9)
    layers1 = trial.suggest_int("layers1", 0, 3)
    layers2 = trial.suggest_int("layers2", 0, 3)
    ln = trial.suggest_int("layer_norms", 0, 2)
    batch_size = trial.suggest_int("batch_size", 8, 1024)
    noise = trial.suggest_int("noise", 0, 1)

    model = getModel(neurons0=neurons0,
                     neurons1=neurons1,
                     neurons2=neurons2,
                     layers1=layers1,
                     layers2=layers2,
                     dropout=dropout,
                     lr=lr,
                     ln=ln,
                     noise=noise)
    
    history = model.fit(X/10.0,y,
              validation_data=(X_val/10.0,y_val),
              batch_size=batch_size,
              epochs=max_epochs,
              verbose=0,
             callbacks=[tf.keras.callbacks.TensorBoard(
                            log_dir="logs/optuna2/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                            histogram_freq=1,
                            write_graph=True,
                            update_freq="epoch"),
                        TFKerasPruningCallback(trial, "val_my_f1"),
                        tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_my_f1",
        factor=0.3,
        patience=10,
        verbose=0,
        mode="max",
        min_delta=0.0001,
        cooldown=0,
        min_lr=1e-10,
    )])
    
    return max(history.history['val_my_f1'])

In [31]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.PatientPruner(optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=max_epochs, reduction_factor=3), patience=10))
study.optimize(objective, callbacks=[TensorBoardCallback('optuna-keras-logs2','val_my_f1')])

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

<ipython-input-31-77d3958e3b70>:2: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.HyperbandPruner(
[I 2022-02-08 22:58:26,437] A new study created in memory with name: no-name-f4de1cc7-a0e4-4f62-ae52-3ba3e845d81e
<ipython-input-31-77d3958e3b70>:4: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
  study.optimize(objective, callbacks=[TensorBoardCallback('optuna-keras-logs2','val_my_f1')])
[I 2022-02-08 23:31:24,239] Trial 0 finished with value: 0.19902470707893372 and parameters: {'neurons0': 122, 'neurons1': 1517, 'neurons2': 38, 'lr': 0.041744602692887586, 'dropout': 0.8409820791659567, 'layers1': 3, 'layers2': 3, 'layer_norms': 1, 'batch_size': 30, 'noise': 0}. Best is trial 0 with value: 0.19902470707893372.
[I 2022-02-08 23:33:14,282] Trial 1 finished with value: 0.7856473922729492 and par

[I 2022-02-09 01:12:28,307] Trial 49 pruned. Trial was pruned at epoch 34.
[I 2022-02-09 01:13:15,135] Trial 50 pruned. Trial was pruned at epoch 40.
[I 2022-02-09 01:14:18,384] Trial 51 pruned. Trial was pruned at epoch 50.
[I 2022-02-09 01:15:25,959] Trial 52 pruned. Trial was pruned at epoch 68.
[I 2022-02-09 01:16:29,684] Trial 53 finished with value: 0.7902214527130127 and parameters: {'neurons0': 472, 'neurons1': 449, 'neurons2': 24, 'lr': 0.0006656531551203954, 'dropout': 0.6909650105143859, 'layers1': 0, 'layers2': 3, 'layer_norms': 0, 'batch_size': 1020, 'noise': 0}. Best is trial 33 with value: 0.8190227150917053.
[I 2022-02-09 01:17:09,355] Trial 54 pruned. Trial was pruned at epoch 49.
[I 2022-02-09 01:18:48,482] Trial 55 finished with value: 0.8157367706298828 and parameters: {'neurons0': 547, 'neurons1': 1355, 'neurons2': 32, 'lr': 2.1979996785471997e-05, 'dropout': 0.462770065893936, 'layers1': 0, 'layers2': 3, 'layer_norms': 0, 'batch_size': 634, 'noise': 0}. Best is tr

[I 2022-02-09 02:50:58,332] Trial 125 pruned. Trial was pruned at epoch 61.
[I 2022-02-09 02:52:06,945] Trial 126 pruned. Trial was pruned at epoch 21.
[I 2022-02-09 02:52:44,439] Trial 127 pruned. Trial was pruned at epoch 30.
[I 2022-02-09 02:53:11,689] Trial 128 pruned. Trial was pruned at epoch 33.
[I 2022-02-09 03:01:06,612] Trial 129 pruned. Trial was pruned at epoch 18.
[I 2022-02-09 03:01:39,347] Trial 130 pruned. Trial was pruned at epoch 40.
[I 2022-02-09 03:05:40,693] Trial 131 pruned. Trial was pruned at epoch 45.
[I 2022-02-09 03:15:20,209] Trial 132 pruned. Trial was pruned at epoch 69.
[I 2022-02-09 03:15:59,921] Trial 133 pruned. Trial was pruned at epoch 34.
[I 2022-02-09 03:22:15,917] Trial 134 pruned. Trial was pruned at epoch 81.
[I 2022-02-09 03:23:22,541] Trial 135 pruned. Trial was pruned at epoch 60.
[I 2022-02-09 03:33:43,217] Trial 136 pruned. Trial was pruned at epoch 52.
[I 2022-02-09 03:34:28,619] Trial 137 pruned. Trial was pruned at epoch 33.
[I 2022-02-0

AssertionError: Concurrent access?

In [37]:

neurons0 = 642
neurons1 = 434
neurons2 = 38
lr = 0.0012261
dropout = 0.73423
layers1 = 0
layers2 = 3
ln = 0
batch_size = 963
noise = 1

model = getModel(neurons0=neurons0,
                 neurons1=neurons1,
                 neurons2=neurons2,
                 layers1=layers1,
                 layers2=layers2,
                 dropout=dropout,
                 lr=lr,
                 ln=ln,
                 noise=noise)

history = model.fit(X/10.0,y,
          validation_data=(X_val/10.0,y_val),
          batch_size=batch_size,
          epochs=500,
          verbose=1,
         callbacks=[tf.keras.callbacks.TensorBoard(
                        log_dir="logs/optuna2/best/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                        histogram_freq=1,
                        write_graph=True,
                        update_freq="epoch"),
                    tf.keras.callbacks.EarlyStopping(
                                    monitor="val_my_f1",
                                    patience=50,
                                    verbose=1,
                                    mode="max",
                                    restore_best_weights=True,
                                ),
                    tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_my_f1",
    factor=0.3,
    patience=10,
    verbose=1,
    mode="max",
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-10,
)])


Epoch 1/500
6/6 [==============================] - 1s 160ms/step - loss: 1.0993 - mae: 0.4439 - mse: 0.2224 - accuracy: 0.3324 - my_f1: 0.3186 - my_acc: 0.3393 - val_loss: 1.0815 - val_mae: 0.4406 - val_mse: 0.2185 - val_accuracy: 0.4870 - val_my_f1: 0.5245 - val_my_acc: 0.5243 - lr: 0.0010
Epoch 2/500
6/6 [==============================] - 1s 161ms/step - loss: 1.0753 - mae: 0.4372 - mse: 0.2174 - accuracy: 0.3788 - my_f1: 0.3792 - my_acc: 0.3831 - val_loss: 1.0311 - val_mae: 0.4281 - val_mse: 0.2077 - val_accuracy: 0.6090 - val_my_f1: 0.6034 - val_my_acc: 0.6129 - lr: 0.0010
Epoch 3/500
6/6 [==============================] - 1s 164ms/step - loss: 1.0408 - mae: 0.4264 - mse: 0.2104 - accuracy: 0.4226 - my_f1: 0.4291 - my_acc: 0.4312 - val_loss: 0.9797 - val_mae: 0.4135 - val_mse: 0.1976 - val_accuracy: 0.6125 - val_my_f1: 0.6122 - val_my_acc: 0.6153 - lr: 0.0010
Epoch 4/500
6/6 [==============================] - 1s 156ms/step - loss: 1.0016 - mae: 0.4125 - mse: 0.2036 - accuracy: 0.44

Epoch 29/500
6/6 [==============================] - 1s 158ms/step - loss: 0.6445 - mae: 0.2708 - mse: 0.1304 - accuracy: 0.7274 - my_f1: 0.7272 - my_acc: 0.7299 - val_loss: 0.5521 - val_mae: 0.2433 - val_mse: 0.1113 - val_accuracy: 0.7505 - val_my_f1: 0.7588 - val_my_acc: 0.7649 - lr: 0.0010
Epoch 30/500
6/6 [==============================] - 1s 195ms/step - loss: 0.6372 - mae: 0.2674 - mse: 0.1282 - accuracy: 0.7422 - my_f1: 0.7401 - my_acc: 0.7434 - val_loss: 0.5460 - val_mae: 0.2381 - val_mse: 0.1100 - val_accuracy: 0.7495 - val_my_f1: 0.7638 - val_my_acc: 0.7684 - lr: 0.0010
Epoch 31/500
6/6 [==============================] - 1s 184ms/step - loss: 0.6433 - mae: 0.2668 - mse: 0.1296 - accuracy: 0.7262 - my_f1: 0.7174 - my_acc: 0.7230 - val_loss: 0.5429 - val_mae: 0.2367 - val_mse: 0.1095 - val_accuracy: 0.7575 - val_my_f1: 0.7792 - val_my_acc: 0.7822 - lr: 0.0010
Epoch 32/500
6/6 [==============================] - 1s 157ms/step - loss: 0.6327 - mae: 0.2620 - mse: 0.1263 - accuracy: 

6/6 [==============================] - 1s 150ms/step - loss: 0.5753 - mae: 0.2430 - mse: 0.1170 - accuracy: 0.7618 - my_f1: 0.7568 - my_acc: 0.7604 - val_loss: 0.5133 - val_mae: 0.2201 - val_mse: 0.1036 - val_accuracy: 0.7750 - val_my_f1: 0.7788 - val_my_acc: 0.7819 - lr: 3.0000e-04
Epoch 57/500
6/6 [==============================] - ETA: 0s - loss: 0.5749 - mae: 0.2430 - mse: 0.1158 - accuracy: 0.7678 - my_f1: 0.7609 - my_acc: 0.7641
Epoch 57: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
6/6 [==============================] - 1s 151ms/step - loss: 0.5749 - mae: 0.2430 - mse: 0.1158 - accuracy: 0.7678 - my_f1: 0.7609 - my_acc: 0.7641 - val_loss: 0.5146 - val_mae: 0.2184 - val_mse: 0.1037 - val_accuracy: 0.7755 - val_my_f1: 0.7824 - val_my_acc: 0.7864 - lr: 3.0000e-04
Epoch 58/500
6/6 [==============================] - 1s 149ms/step - loss: 0.5772 - mae: 0.2409 - mse: 0.1163 - accuracy: 0.7678 - my_f1: 0.7613 - my_acc: 0.7663 - val_loss: 0.5118 - val_mae: 0.2184 - 

6/6 [==============================] - 1s 148ms/step - loss: 0.5652 - mae: 0.2376 - mse: 0.1135 - accuracy: 0.7732 - my_f1: 0.7789 - my_acc: 0.7834 - val_loss: 0.5101 - val_mae: 0.2161 - val_mse: 0.1030 - val_accuracy: 0.7710 - val_my_f1: 0.7755 - val_my_acc: 0.7791 - lr: 8.1000e-06
Epoch 82/500
6/6 [==============================] - 1s 176ms/step - loss: 0.5687 - mae: 0.2396 - mse: 0.1145 - accuracy: 0.7750 - my_f1: 0.7827 - my_acc: 0.7864 - val_loss: 0.5101 - val_mae: 0.2160 - val_mse: 0.1030 - val_accuracy: 0.7710 - val_my_f1: 0.7755 - val_my_acc: 0.7791 - lr: 8.1000e-06
Epoch 83/500
6/6 [==============================] - 1s 165ms/step - loss: 0.5751 - mae: 0.2400 - mse: 0.1160 - accuracy: 0.7606 - my_f1: 0.7524 - my_acc: 0.7572 - val_loss: 0.5101 - val_mae: 0.2160 - val_mse: 0.1030 - val_accuracy: 0.7705 - val_my_f1: 0.7799 - val_my_acc: 0.7829 - lr: 8.1000e-06
Epoch 84/500
6/6 [==============================] - 1s 152ms/step - loss: 0.5810 - mae: 0.2403 - mse: 0.1164 - accuracy: 0

In [ ]:
# model.fit(X/10.0,y,
#           validation_data=(X_val/10.0,y_val),
#           batch_size=1024,
#           epochs=500,
#           verbose=0,
#          callbacks=[tf.keras.callbacks.TensorBoard(
#     log_dir="logs/0/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
#     histogram_freq=1,
#     write_graph=True,
#     update_freq="epoch"),#PlotLossesKerasTF(),
# #                    tf.keras.callbacks.ModelCheckpoint(
# #     'checkpoint',
# #     monitor="val_my_f1",
# #     verbose=1,
# #     save_best_only=True,
# #     save_weights_only=False,
# #     mode="max",
# #     save_freq="epoch",),
#                     tf.keras.callbacks.EarlyStopping(
#                                     monitor="val_my_f1",
#                                     patience=100,
#                                     verbose=1,
#                                     mode="max",
#                                     restore_best_weights=True,
#                                 ),
#                    tf.keras.callbacks.ReduceLROnPlateau(
#     monitor="val_my_f1",
#     factor=0.3,
#     patience=10,
#     verbose=1,
#     mode="max",
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=1e-10,
# )])

In [45]:
y_pred = model.predict(X_val/10.0)
# y_pred.shape
print(f"F1 Score : {f1_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1), average='weighted')}")
print(f"Accuracy Score : {accuracy_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1))}")

F1 Score : 0.7637943535908992
Accuracy Score : 0.761


In [46]:
y_val[:10],y_pred[:10]

(<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
 array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.]], dtype=float32)>,
 array([[1.4942489e-03, 2.8291926e-01, 7.1558642e-01],
        [1.0094841e-01, 7.2233212e-01, 1.7671947e-01],
        [8.6465949e-01, 1.3228248e-01, 3.0580773e-03],
        [1.8148777e-01, 7.6399928e-01, 5.4512929e-02],
        [7.9855815e-02, 6.7361981e-01, 2.4652432e-01],
        [8.7316757e-01, 1.2421182e-01, 2.6207031e-03],
        [7.4558127e-01, 2.4679761e-01, 7.6211370e-03],
        [2.4400548e-04, 1.7287822e-01, 8.2687777e-01],
        [7.9243368e-01, 2.0136975e-01, 6.1966032e-03],
        [9.7946683e-03, 5.5327481e-01, 4.3693057e-01]], dtype=float32))

In [40]:
np.count_nonzero(y_pred)

6000

## Generating the Predictions

Generating Predictions from test data to make submission in the puzzle. 

In [41]:
submission_embeddings = [literal_eval(embedding)  for embedding in submission['embeddings'].values]

predictions = np.argmax(model.predict(np.array(submission_embeddings)/10.0),axis=1).tolist()

In [42]:
def convert(a):
    if a==0:
        return 'negative'
    if a==1:
        return 'neutral'
    if a==2:
        return 'positive'

result = list(map(convert, predictions))

In [43]:
np.unique(result, return_counts=True)

(array(['negative', 'neutral', 'positive'], dtype='<U8'),
 array([1004, 1083,  914]))

In [48]:
submission['label'] = result
submission

,embeddings,label
0,"[0.08109518140554428, 0.3090009093284607, 1.36...",positive
1,"[0.6809610724449158, 1.1909409761428833, 0.892...",neutral
2,"[0.14851869642734528, 0.7872061133384705, 0.89...",neutral
3,"[0.44697386026382446, 0.36429283022880554, 0.7...",neutral
4,"[1.8009324073791504, 0.26081395149230957, 0.40...",negative
...,...,...
2996,"[0.9138844609260559, 0.9460961222648621, 0.571...",negative
2997,"[0.7667452096939087, 0.7896291613578796, 0.648...",negative
2998,"[0.8158280849456787, 2.404792070388794, 0.9924...",neutral
2999,"[0.4161085784435272, 0.3146701455116272, 1.139...",positive


### Saving the Predictions

In [49]:
# Saving the predictions
!rm -rf assets
!mkdir assets
submission.to_csv(os.path.join("assets", "submission.csv"))

## Submitting our Predictions

In [50]:
%load_ext aicrowd.magic

In [51]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/Anb7h1ae69c1cBsDUoI8ZfoBk9mkoiU_YJUACGldK5M
API Key valid
Gitlab access token valid
Saved details successfully!


In [52]:
%aicrowd notebook submit -c sentiment-classification -a assets --no-verify

<IPython.core.display.Javascript object>

Using notebook: keras-mlp.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
An unexpected error occured!
cannot unpack non-iterable NoneType object
To get more information, you can run this command with -v.
To increase level of verbosity, you can go upto -vvvvv


In [ ]:
!ls assets

In [55]:
! cd assets/ ; zip submission.zip submission.csv

  adding: submission.csv (deflated 56%)


Congratulations to making your first submission in the puzzle 🎉  . Let's continue with the journey by improving the baseline & making submission! Don't be shy to ask question related to any errors you are getting or doubts in any part of this notebook in discussion forum or in AIcrowd Discord sever, AIcrew will be happy to help you :)

Have a cool new idea that you want to see in the next blitz ? Let us know!